In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from evaluation.analysis import Plotter
plt = Plotter("D:/Guy_Gilad/Evaluation/Eval_v2/bboxes.csv")
plt.plot_v1()

In [ ]:
data = plt._data
mask = data['bbox_area_diff'] <= 0.2
data[mask].describe()

In [ ]:
plt._data = data[mask]
plt.plot_v1()

In [ ]:
import seaborn as sns
data = plt._data
data['wrm_speed_avg'] = data['wrm_speed'].rolling(window=5, center=True).mean()
ax = sns.scatterplot(data=plt._data, x='wrm_speed_avg', y='bbox_area_diff')
ax.set_xlim(0,10)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from evaluation.simulator import *
from evaluation.sim_controllers import *
from utils.path_utils import Files

time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.225, 0.225),
    init_position=(1300, 1200),
    frame_padding_value=(255, 255, 255),
)

log_config = LogConfig(
    root_folder="D:/Guy_Gilad/Evaluation/Eval_v2_advanced",
    save_mic_view=True,
    save_cam_view=True,
)

log_config.create_dirs()

yolo_config = YoloConfig(model_path="yolov8m-trained.pt", verbose=True) 

time_config.save_json(log_config.root_folder+"/time_config.json")
log_config.save_json(log_config.root_folder+"/log_config.json")
yolo_config.save_json(log_config.root_folder+"/yolo_config.json")


frames_path = "D:/Guy_Gilad/RawData/Exp_20240213_N2_JC01"
files = Files(frames_path, extension='Bmp', sorting_func=lambda f: int(f.split('-')[-1].split('.')[0]))
files_list = [file for file in files]
reader = FrameReader(frames_path, files_list)

# time_config = TimingConfig.load_json("logs/time_config.json")
# log_config = LogConfig.load_json("logs/log_config.json")
# yolo_config = YoloConfig.load_json("logs/yolo_config.json")


In [ ]:
print(time_config.imaging_frame_num + time_config.moving_frame_num)

In [ ]:
# controller = CsvController(time_config, "D:/Guy_Gilad/Evaluation/Eval_v2/bboxes.csv")
# controller = LoggingController(time_config, yolo_config, log_config)
controller = YoloController(time_config, yolo_config)
sim = Simulator(time_config, reader, controller)

In [ ]:
sim.run(visualize=True)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes